In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



In [11]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range=0.2,
                                   shear_range= 0.2,
                                   horizontal_flip= True,
                                   vertical_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

train_dir_path = "ocr-data/data/training_data/"
valid_dir_path = "ocr-data/data/testing_data"

train_data = train_datagen.flow_from_directory(train_dir_path,
                                               target_size = (150,150),
                                               class_mode='categorical',
                                               seed = 42)

valid_data = valid_datagen.flow_from_directory(valid_dir_path,
                                               target_size = (150,150),
                                               class_mode = 'categorical',
                                               seed = 42)


Found 20628 images belonging to 36 classes.
Found 1008 images belonging to 36 classes.


In [5]:
from pathlib import Path
class_name = np.array(sorted(item.name for item in Path(train_dir_path).glob("*")))
print(class_name)

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [6]:
len(class_name)

36

In [12]:
model_1 = Sequential([
    layers.Conv2D(16,3, activation = "relu", input_shape = (150,150,2)),
    layers.Conv2D(32,3,activation = "relu"),
    layers.MaxPool2D(2),
    layers.Conv2D(64,3, activation = "relu"),
    layers.MaxPool2D(2),
    layers.Conv2D(64,3, activation = "relu"),
    layers.MaxPool2D(2),
    layers.Conv2D(64,3, activation = "relu"),
    layers.MaxPool2D(2),
    layers.Flatten(),
    layers.Dense(300, activation = "relu"),
    layers.Dense(36, activation = "softmax")
])

model_1.compile(loss = "categorical_crossentropy",
                optimizer = "rmsprop",
                metrics = ["accuracy"])

history_1 = model_1.fit(train_data,
                      epochs = 10,
                      steps_per_epoch = len(train_data),
                      validation_data = valid_data,
                      validation_steps = len(valid_data))

Epoch 1/10
645/645 [==============================] - 409s 633ms/step - loss: 3.0324 - accuracy: 0.1428 - val_loss: 1.5543 - val_accuracy: 0.4563
Epoch 2/10
645/645 [==============================] - 305s 472ms/step - loss: 1.8904 - accuracy: 0.3968 - val_loss: 0.8578 - val_accuracy: 0.7024
Epoch 3/10
645/645 [==============================] - 308s 478ms/step - loss: 1.3458 - accuracy: 0.5623 - val_loss: 0.4564 - val_accuracy: 0.8423
Epoch 4/10
645/645 [==============================] - 317s 492ms/step - loss: 1.0643 - accuracy: 0.6468 - val_loss: 0.3366 - val_accuracy: 0.8720
Epoch 5/10
645/645 [==============================] - 320s 496ms/step - loss: 0.9080 - accuracy: 0.7016 - val_loss: 0.3275 - val_accuracy: 0.8552
Epoch 6/10
645/645 [==============================] - 318s 492ms/step - loss: 0.7904 - accuracy: 0.7400 - val_loss: 0.2774 - val_accuracy: 0.9067
Epoch 7/10
645/645 [==============================] - 319s 495ms/step - loss: 0.7236 - accuracy: 0.7628 - val_loss: 0.3068 -

In [13]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 conv2d_6 (Conv2D)           (None, 146, 146, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 35, 35, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 33, 33, 64)       

In [76]:
def load_and_prep_data(filename, filesize = [150,150]):
  # read the images
  img = tf.io.read_file(filename)
  # decode the images
  img = tf.io.decode_image(contents=img, channels = 3)
  # resize the image 
  img = tf.image.resize(img , size = filesize)
  img = img/255.
  # expand the dims
  img = tf.expand_dims(img, axis = 0)
  

  return img

     

In [77]:
def evaluate(img):
    f = np.round(img_class)

    for i in range(len(f[0])):
        if f[0][i] == 1:
            print(i)
            return i

In [101]:
exp_path = "j.png"


In [102]:
image = load_and_prep_data(exp_path)
img_class =  model_1.predict(image)
f_image= evaluate(img_class)


1/1 [==============================] - 0s 22ms/step
19


In [103]:
class_name[f_image]


'J'